In [26]:
from keras.preprocessing import sequence, text
from keras.models import Sequential
from keras.layers import Dense, Embedding
from keras.layers import LSTM
import pandas as pd
import numpy as np

#train_data_source = './exercise_data/human_dna_train_small.csv'
train_data_source = './exercise_data/human_dna_train_split_5_95.csv'
val_data_source = './exercise_data/human_dna_validation_split.csv'
test_data_source = './exercise_data/human_dna_test_split.csv'

train_df = pd.read_csv(train_data_source, header=0)
test_df = pd.read_csv(test_data_source, header=0)

train_seq = train_df['sequences']
train_label = train_df['labels']
test_seq = test_df['sequences']
test_label = test_df['labels']

# Preprocess
tk = text.Tokenizer(char_level=True)
tk.fit_on_texts(train_seq)

train_seq_tok = tk.texts_to_sequences(train_seq)
test_seq_tok = tk.texts_to_sequences(test_seq)

train_seq = np.array(train_seq)
train_seq_tok = np.array(train_seq_tok)
train_label = np.array(train_label)
test_seq = np.array(test_seq)
test_seq_tok = np.array(test_seq_tok)
test_label = np.array(test_label)

print('train_seq shape:', train_seq.shape)
print('test_seq shape:', test_seq.shape)

train_seq shape: (29420,)
test_seq shape: (33333,)


In [27]:
train_label[train_label<0]=0
test_label[test_label<0]=0

In [19]:
train_label

array([1, 1, 1, ..., 0, 0, 0])

In [28]:
train_seq_tok = train_seq_tok - np.ones(shape=train_seq_tok.shape)
test_seq_tok = test_seq_tok - np.ones(shape=test_seq_tok.shape)

In [29]:
train_seq_tok[0]

array([0., 3., 1., 1., 2., 1., 2., 2., 0., 0., 1., 1., 2., 3., 1., 1., 3.,
       0., 0., 2., 0., 2., 2., 1., 0., 2., 2., 0., 3., 1., 0., 2., 2., 1.,
       3., 2., 1., 0., 1., 2., 2., 2., 1., 1., 3., 0., 1., 0., 0., 2., 3.,
       1., 1., 1., 0., 0., 3., 0., 0., 2., 1., 2., 3., 1., 0., 2., 2., 0.,
       2., 0., 0., 2., 1., 3., 1., 2., 2., 2., 1., 1., 2., 1., 1., 1., 1.,
       3., 1., 2., 0., 1., 0., 0., 3., 3., 1., 2., 1., 0., 3., 1., 2., 0.,
       0., 1., 2., 0., 2., 0., 2., 2., 3., 1., 2., 0., 1., 1., 1., 1., 3.,
       1., 3., 1., 0., 2., 1., 2., 2., 2., 2., 0., 2., 1., 3., 0., 2., 1.,
       1., 2., 2., 2., 1., 1., 2., 3., 1., 2., 3., 1., 1., 1., 1., 1., 3.,
       0., 2., 2., 3., 1., 2., 1., 1., 1., 1., 1., 1., 3., 1., 1., 2., 3.,
       1., 2., 3., 0., 3., 0., 0., 0., 3., 0., 3., 0., 2., 0., 0., 0., 0.,
       0., 0., 0., 0., 0., 2., 3., 3., 1., 1., 1., 2., 2., 1., 0., 1., 3.,
       3., 0., 3., 3., 3., 0., 3., 1., 1., 3., 1., 1., 0., 2., 2., 2., 3.,
       1., 0., 0., 3., 3.

In [33]:
from keras.layers import Dropout
import keras as K

# def recall_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     possible_positives = K.sum(K.round(K.clip(y_true, 0, 1)))
#     recall = true_positives / (possible_positives + K.epsilon())
#     return recall

# def precision_m(y_true, y_pred):
#     true_positives = K.sum(K.round(K.clip(y_true * y_pred, 0, 1)))
#     predicted_positives = K.sum(K.round(K.clip(y_pred, 0, 1)))
#     precision = true_positives / (predicted_positives + K.epsilon())
#     return precision

# def f1_m(y_true, y_pred):
#     precision = precision_m(y_true, y_pred)
#     recall = recall_m(y_true, y_pred)
#     return 2*((precision*recall)/(precision+recall+K.epsilon()))

# max_features = 20000
batch_size = 256
epochs = 100
learning_rate = 0.001

model = Sequential()
model.add(Dense(300, activation='relu', input_shape=(398,)))
model.add(Dropout(0.2))
model.add(Dense(250, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(200, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(150, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(100, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(50, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(20, activation='relu'))
# model.add(Dropout(0.2))
model.add(Dense(1, activation='sigmoid'))

# try using different optimizers and different optimizer configs
opt = K.optimizers.Adam(learning_rate=learning_rate)
model.compile(loss='binary_crossentropy', optimizer=opt, metrics=['acc'])

print('Train...')
model.fit(train_seq_tok, train_label, batch_size=batch_size, epochs=epochs, validation_data=(test_seq_tok, test_label))
score, acc = model.evaluate(test_seq_tok, test_label, batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...
Train on 29420 samples, validate on 33333 samples
Epoch 1/100
29420/29420 [==============================] - 9s 292us/step - loss: 0.2116 - acc: 0.9492 - val_loss: 0.0731 - val_acc: 0.9969
Epoch 2/100
29420/29420 [==============================] - 8s 281us/step - loss: 0.1885 - acc: 0.9500 - val_loss: 0.0416 - val_acc: 0.9969
Epoch 3/100
29420/29420 [==============================] - 8s 279us/step - loss: 0.1834 - acc: 0.9500 - val_loss: 0.0975 - val_acc: 0.9969
Epoch 4/100
29420/29420 [==============================] - 8s 284us/step - loss: 0.1789 - acc: 0.9500 - val_loss: 0.0459 - val_acc: 0.9969
Epoch 5/100
29420/29420 [==============================] - 8s 282us/step - loss: 0.1760 - acc: 0.9499 - val_loss: 0.0451 - val_acc: 0.9969
Epoch 6/100
29420/29420 [==============================] - 8s 277us/step - loss: 0.1734 - acc: 0.9500 - val_loss: 0.0991 - val_acc: 0.9969
Epoch 7/100
29420/29420 [==============================] - 8s 277us/step - loss: 0.1710 - acc: 0.9500 - val

29420/29420 [==============================] - 8s 280us/step - loss: 0.0847 - acc: 0.9674 - val_loss: 0.0508 - val_acc: 0.9915
Epoch 60/100
29420/29420 [==============================] - 8s 271us/step - loss: 0.0803 - acc: 0.9682 - val_loss: 0.0541 - val_acc: 0.9898
Epoch 61/100
29420/29420 [==============================] - 8s 276us/step - loss: 0.0770 - acc: 0.9687 - val_loss: 0.0450 - val_acc: 0.9949
Epoch 62/100
29420/29420 [==============================] - 8s 275us/step - loss: 0.0825 - acc: 0.9672 - val_loss: 0.0666 - val_acc: 0.9901
Epoch 63/100
29420/29420 [==============================] - 8s 273us/step - loss: 0.0781 - acc: 0.9685 - val_loss: 0.0590 - val_acc: 0.9899
Epoch 64/100
29420/29420 [==============================] - 8s 278us/step - loss: 0.0765 - acc: 0.9688 - val_loss: 0.0802 - val_acc: 0.9864
Epoch 65/100
29420/29420 [==============================] - 8s 283us/step - loss: 0.0782 - acc: 0.9678 - val_loss: 0.0524 - val_acc: 0.9904
Epoch 66/100
29420/29420 [=======